# EXPERIMENT NUMBER 2

Risk function is defined as:
$$
 R= \sum_{i=1}^{n} \frac{R_{i}}{n}
$$
where $R_i$ is defined as the maximum increase in happiness among all manipulations in set $J$
 $$
 R_i = \max\{ \Delta H_j \mid j \in J \}
 $$


It would be interesting to analyze the variation of the strategic risk as a function of the number of voters, the number of candidates and the voting scheme. 

In [1]:
from src.utils import random_voting
import numpy as np
from src.happiness_level import HappinessLevel
from src.outcomes import  plurality_outcome, borda_outcome, for_two_outcome, veto_outcome
from src.strategic_voting_risk import StrategicVoting
from src.utils import VotingSchemas
import matplotlib.pyplot as plt
from typing import Callable
from itertools import permutations
import pandas as pd
import math
from numba import prange

In [2]:
class HappinessLevelTest:
    def __init__(self, preferences: np.ndarray, voting_schema: str):
        self.preferences = preferences
        self.voting_schema = voting_schema
        self.k = 0.95
        self.c = 1 / (2 * math.atanh(self.k))

    def happiness_level(self, vwr: int) -> float:
        h_i = 1 - 2 / (self.preferences.shape[0] - 1) * vwr
        h = math.atanh(h_i * self.k) * self.c + 0.5
        return h

class TestStrategicVoting2:
    def __init__(self, preferences: np.ndarray, happiness: HappinessLevelTest, schema_outcome_f: Callable):
        self.preferences = preferences
        self.happiness = happiness
        self.schema_outcome_f = schema_outcome_f
        self.happinesses = np.tile(happiness.voter, (2, preferences.shape[1]))
        self.risk = self._compute_risk()

    def _compute_risk(self) -> float:
        result = self.schema_outcome_f(self.preferences)
        voter_count = self.preferences.shape[1]
        total_risk = 0
        
        for i in prange(voter_count):
            vwr = np.argwhere(self.preferences[:, i] == result.winner)[0][0]
            max_voter_happiness = self.happinesses[1, i]
            
            for p in permutations(self.preferences[:, i]):
                p = np.array(p)
                p_winner_index = np.argwhere(p == result.winner)[0][0]
                
                if p_winner_index >= vwr:
                    new_voting = self.preferences.copy()
                    new_voting[:, i] = p
                    new_result = self.schema_outcome_f(new_voting)
                    new_vwr = np.argwhere(new_voting[:, i] == new_result.winner)[0][0]
                    voter_happiness = self.happiness.happiness_level(new_vwr)
                    self.happinesses[1, i] = max(max_voter_happiness, voter_happiness)
            
            total_risk += self.happinesses[1, i] - self.happinesses[0, i]
        
        num_unhappy_voters = np.count_nonzero(self.happiness.voter != 1)
        return total_risk / num_unhappy_voters if num_unhappy_voters != 0 else 0


In [3]:
voters = 10
candidates = 5
min_candidates = 2

In [4]:
# Initialize lists to store risk values for each scheme and number of candidates
risk_values_plurality_list = []
risk_values_fortwo_list = []
risk_values_veto_list = []
risk_values_borda_list = []

num_candidates_range = prange(min_candidates, candidates + 1)

# Iterate over different numbers of candidates
for candidates_current in num_candidates_range:
    risk_values_plurality = np.zeros(voters - 1)  # Initialize array to store risk values
    risk_values_fortwo = np.zeros(voters - 1)
    risk_values_veto = np.zeros(voters - 1)
    risk_values_borda = np.zeros(voters - 1)

    # Generate all random votes for the current number of candidates
    all_random_votes = random_voting(voters, candidates_current)

    # Iterate over different numbers of voters
    for num_voters in prange(2, voters + 1):
        # Slice the random votes array to get current voting situation
        voting_situation = all_random_votes[:, :num_voters]

        # Compute outcomes, happiness, and risk for Plurality
        outcomes = plurality_outcome(voting_situation)
        happiness = HappinessLevel(voting_situation, outcomes.winner, "PLURALITY").run()
        risk_values_plurality[num_voters - 2] = TestStrategicVoting2(voting_situation, happiness, plurality_outcome).risk
        
        # Compute outcomes, happiness, and risk for For Two
        outcomes = for_two_outcome(voting_situation)
        happiness = HappinessLevel(voting_situation, outcomes.winner, "FOR_TWO").run()
        risk_values_fortwo[num_voters - 2] = TestStrategicVoting2(voting_situation, happiness, for_two_outcome).risk
        
        # Compute outcomes, happiness, and risk for Veto
        outcomes = veto_outcome(voting_situation)
        happiness = HappinessLevel(voting_situation, outcomes.winner, "VETO").run()
        risk_values_veto[num_voters - 2] = TestStrategicVoting2(voting_situation, happiness, veto_outcome).risk
        
        # Compute outcomes, happiness, and risk for Borda
        outcomes = borda_outcome(voting_situation)
        happiness = HappinessLevel(voting_situation, outcomes.winner, "BORDA").run()
        risk_values_borda[num_voters - 2] = TestStrategicVoting2(voting_situation, happiness, borda_outcome).risk

    # Append risk values to the respective lists
    risk_values_plurality_list.append(risk_values_plurality)
    risk_values_fortwo_list.append(risk_values_fortwo)
    risk_values_veto_list.append(risk_values_veto)
    risk_values_borda_list.append(risk_values_borda)

# Plotting
num_voters_range = np.arange(2, voters + 1)

# Plot for Plurality and For Two
plt.figure(figsize=(10, 6))
for i, num_candidates in enumerate(num_candidates_range):
    plt.subplot(2, 2, 1)
    plt.plot(num_voters_range, risk_values_plurality_list[i], label=f'{num_candidates} Candidates')
    plt.xlabel('Number of Voters')
    plt.ylabel('Risk')
    plt.title('Plurality')
    plt.legend(title='Number of Candidates')
    plt.grid(True)

    plt.subplot(2, 2, 2)
    plt.plot(num_voters_range, risk_values_fortwo_list[i], label=f'{num_candidates} Candidates')
    plt.xlabel('Number of Voters')
    plt.ylabel('Risk')
    plt.title('For Two')
    plt.legend(title='Number of Candidates')
    plt.grid(True)

    if (i + 1) % 2 == 0 or i == len(num_candidates_range) - 1:
        plt.tight_layout()
        plt.show()

# Plot for Veto and Borda
plt.figure(figsize=(10, 6))
for i, num_candidates in enumerate(num_candidates_range):
    plt.subplot(2, 2, 1)
    plt.plot(num_voters_range, risk_values_veto_list[i], label=f'{num_candidates} Candidates')
    plt.xlabel('Number of Voters')
    plt.ylabel('Risk')
    plt.title('Veto')
    plt.legend(title='Number of Candidates')
    plt.grid(True)

    plt.subplot(2, 2, 2)
    plt.plot(num_voters_range, risk_values_borda_list[i], label=f'{num_candidates} Candidates')
    plt.xlabel('Number of Voters')
    plt.ylabel('Risk')
    plt.title('Borda')
    plt.legend(title='Number of Candidates')
    plt.grid(True)

    if (i + 1) % 2 == 0 or i == len(num_candidates_range) - 1:
        plt.tight_layout()
        plt.show()

KeyboardInterrupt: 